In [44]:
import requests
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Optional

# ----------------------
# Pydantic Schemas
# ----------------------
class PlayerInput(BaseModel):
    name: Optional[str] = Field(None, description="Name of the player (can be null)")
    age: Optional[int] = Field(None, description="Age of the player (can be null)")
    sport: Optional[str] = Field(None, description="Sport the player plays (can be null)")

class UpdatePlayerInput(BaseModel):
    player_id: int = Field(..., description="ID of the player to update")
    name: Optional[str] = Field(None, description="Updated name of the player (optional)")
    age: Optional[int] = Field(None, description="Updated age of the player (optional)")
    sport: Optional[str] = Field(None, description="Updated sport of the player (optional)")

class DeletePlayerInput(BaseModel):
    player_id: int = Field(..., description="ID of the player to delete")

class QueryInput(BaseModel):
    query: str = Field(..., description="A raw SQL SELECT query to run on the database")


# ----------------------
# API Wrappers
# ----------------------
def call_mcp_add(name: Optional[str] = None, age: Optional[int] = None, sport: Optional[str] = None):
    """Call API to add a new player. Missing fields will be NULL."""
    payload = {"name": name, "age": age, "sport": sport}
    r = requests.post("http://localhost:8000/add_player", json=payload)
    return r.json()

def call_mcp_read():
    """Call API to read all players."""
    r = requests.get("http://localhost:8000/read_players")
    return r.json()

def call_mcp_update(player_id: int, name: Optional[str] = None, age: Optional[int] = None, sport: Optional[str] = None):
    """Call API to update only provided fields for a player by ID."""
    payload = {}
    if name is not None:
        payload["name"] = name
    if age is not None:
        payload["age"] = age
    if sport is not None:
        payload["sport"] = sport

    r = requests.put(f"http://localhost:8000/update_player/{player_id}", json=payload)
    return r.json()

def call_mcp_delete(player_id: int):
    """Call API to delete a player by ID."""
    r = requests.delete(f"http://localhost:8000/delete_player/{player_id}")
    return r.json()

def call_mcp_query(query: str):
    """Call API to run a raw SQL query (SELECT only)."""
    r = requests.post("http://localhost:8000/run_query", json={"query": query})
    return r.json()


# ----------------------
# LangChain Tools
# ----------------------
mcp_add_tool = StructuredTool.from_function(
    func=call_mcp_add,
    name="add_player",
    description="Add a new player to the SQLite DB. Missing fields will be stored as NULL.",
    args_schema=PlayerInput
)

mcp_read_tool = StructuredTool.from_function(
    func=call_mcp_read,
    name="read_players",
    description="Read all players from the SQLite DB. No input required."
)

mcp_update_tool = StructuredTool.from_function(
    func=call_mcp_update,
    name="update_player",
    description="Update only provided fields of a player by ID.",
    args_schema=UpdatePlayerInput
)

mcp_delete_tool = StructuredTool.from_function(
    func=call_mcp_delete,
    name="delete_player",
    description="Delete a player by ID.",
    args_schema=DeletePlayerInput
)

mcp_query_tool = StructuredTool.from_function(
    func=call_mcp_query,
    name="run_query",
    description="Run a raw SELECT SQL query on the database.",
    args_schema=QueryInput
)


In [48]:
system_prompt = """
You are a database agent specialized in managing a SQLite database of players.

🔒 Rules:
- ONLY interact with the database using the provided tools.
- ALWAYS pick the tool that exactly matches the user request.
- If no tool matches, or the query is complete, STOP and return final answer. Do not keep reasoning endlessly.

"""


In [49]:
from langchain.agents import initialize_agent, AgentType
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="mistral:instruct")

agent = initialize_agent(
    tools=[
        mcp_add_tool,
        mcp_read_tool,
        mcp_update_tool,
        mcp_delete_tool,
        mcp_query_tool
    ],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"system_message": system_prompt},
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
# Add new player
agent.invoke("Update the player with the id=2 increase his age to 22")



> Entering new AgentExecutor chain...


 Thought: The human wants to update a specific player's age in the SQLite DB. I will use the 'update_player' tool for this task.

Action:
```
{
  "action": "update_player",
  "action_input": {
    "player_id": 2,
    "age": 22
  }
}
```

Observation: {'status': 'ok', 'updated_id': 2}
Thought: The action has successfully updated the player with ID 2 to age 22 in the SQLite DB. Now, I need to check if the update was successful by reading all players from the database and checking if player 2's age is now 22.

Action:
```
{
  "action": "read_players",
  "action_input": {}
}
```

Thought: I will check if the player with ID 2 has the correct age after the update. If not, I will need to troubleshoot.


Observation: {'data': [[2, 'John Doe', 22, 'Football'], [3, 'Charlie', 22, 'Basketball'], [4, 'Nguyen Nguyen', 22, 'football']]}
Thought: The player with ID 2 has been successfully updated to age 22 in the SQLite DB. The verification process is complete.

Action:
```
{
  "action": "Final Answe

{'input': 'Update the player with the id=2 increase his age to 22',
 'output': "The player with ID 2's age has been successfully updated to 22."}